# Loop 8 Analysis: Understanding the Gap and Validation Issues

## Key Problems:
1. **Validation mismatch**: 4 consecutive ensemble submissions failed Kaggle validation
2. **Gap to target**: 70.676 vs 68.887 = 1.79 points (2.6%)
3. **Local optimum**: bbox3, SA, fix_direction all found NO improvement on baseline

## Strategy Analysis:
- The baseline (70.676) is the ONLY valid submission
- Ensemble approach finds 0.06 point improvement but fails validation
- Need fundamentally different approach to close 1.79 point gap

In [ ]:
import pandas as pd
import numpy as np
import json
import os

# Load session state
with open('/home/code/session_state.json', 'r') as f:
    state = json.load(f)

# Analyze experiments
print("=" * 60)
print("EXPERIMENT ANALYSIS")
print("=" * 60)

for exp in state['experiments']:
    fallback = exp.get('used_baseline_fallback', False)
    approach_score = exp.get('approach_score', exp['cv_score'])
    print(f"\n{exp['id']}: {exp['name']}")
    print(f"  CV Score: {exp['cv_score']:.6f}")
    print(f"  Approach Score: {approach_score:.6f}")
    print(f"  Fallback to baseline: {fallback}")
    if fallback:
        print(f"  ⚠️ APPROACH FAILED - fell back to baseline")

In [ ]:
# Analyze submissions
print("\n" + "=" * 60)
print("SUBMISSION ANALYSIS")
print("=" * 60)

for sub in state['submissions']:
    lb = sub.get('lb_score')
    error = sub.get('error')
    print(f"\n{sub['experiment_id']}: {sub['model_name']}")
    print(f"  CV: {sub['cv_score']:.6f}")
    if lb:
        print(f"  LB: {lb:.6f} ✅")
    elif error:
        print(f"  LB: FAILED - {error}")
    else:
        print(f"  LB: pending")

In [ ]:
# Calculate theoretical minimum and gap analysis
print("\n" + "=" * 60)
print("GAP ANALYSIS")
print("=" * 60)

best_valid_lb = 70.676102
target = 68.887226
gap = best_valid_lb - target
gap_pct = (gap / target) * 100

print(f"Best valid LB: {best_valid_lb:.6f}")
print(f"Target: {target:.6f}")
print(f"Gap: {gap:.6f} ({gap_pct:.2f}%)")
print()
print("To reach target, we need:")
print(f"  - Reduce score by {gap:.6f} points")
print(f"  - That's {gap / 200:.6f} points per N on average")
print(f"  - Or {gap / 200 / 0.35 * 100:.1f}% improvement per N (assuming avg score ~0.35)")

In [ ]:
# Load baseline and analyze per-N scores
baseline_path = '/home/code/experiments/000_baseline/submission.csv'
baseline_df = pd.read_csv(baseline_path)

print("\n" + "=" * 60)
print("PER-N SCORE ANALYSIS")
print("=" * 60)

from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
import numpy as np

getcontext().prec = 25
scale_factor = Decimal("1e15")

class ChristmasTree:
    def __init__(self, center_x="0", center_y="0", angle="0"):
        self.center_x = Decimal(str(center_x).replace('s', ''))
        self.center_y = Decimal(str(center_y).replace('s', ''))
        self.angle = Decimal(str(angle).replace('s', ''))

        trunk_w = Decimal("0.15")
        trunk_h = Decimal("0.2")
        base_w = Decimal("0.7")
        mid_w = Decimal("0.4")
        top_w = Decimal("0.25")
        tip_y = Decimal("0.8")
        tier_1_y = Decimal("0.5")
        tier_2_y = Decimal("0.25")
        base_y = Decimal("0.0")
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon([
            (float(Decimal("0.0") * scale_factor), float(tip_y * scale_factor)),
            (float(top_w / Decimal("2") * scale_factor), float(tier_1_y * scale_factor)),
            (float(top_w / Decimal("4") * scale_factor), float(tier_1_y * scale_factor)),
            (float(mid_w / Decimal("2") * scale_factor), float(tier_2_y * scale_factor)),
            (float(mid_w / Decimal("4") * scale_factor), float(tier_2_y * scale_factor)),
            (float(base_w / Decimal("2") * scale_factor), float(base_y * scale_factor)),
            (float(trunk_w / Decimal("2") * scale_factor), float(base_y * scale_factor)),
            (float(trunk_w / Decimal("2") * scale_factor), float(trunk_bottom_y * scale_factor)),
            (float(-(trunk_w / Decimal("2")) * scale_factor), float(trunk_bottom_y * scale_factor)),
            (float(-(trunk_w / Decimal("2")) * scale_factor), float(base_y * scale_factor)),
            (float(-(base_w / Decimal("2")) * scale_factor), float(base_y * scale_factor)),
            (float(-(mid_w / Decimal("4")) * scale_factor), float(tier_2_y * scale_factor)),
            (float(-(mid_w / Decimal("2")) * scale_factor), float(tier_2_y * scale_factor)),
            (float(-(top_w / Decimal("4")) * scale_factor), float(tier_1_y * scale_factor)),
            (float(-(top_w / Decimal("2")) * scale_factor), float(tier_1_y * scale_factor)),
        ])
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(
            rotated,
            xoff=float(self.center_x * scale_factor),
            yoff=float(self.center_y * scale_factor),
        )

def load_trees_for_n(df, n):
    prefix = f"{n:03d}_"
    subset = df[df['id'].str.startswith(prefix)]
    trees = []
    for _, row in subset.iterrows():
        x = str(row['x']).replace('s', '')
        y = str(row['y']).replace('s', '')
        deg = str(row['deg']).replace('s', '')
        trees.append(ChristmasTree(x, y, deg))
    return trees

def calculate_score(trees, n):
    xys = np.concatenate([np.asarray(t.polygon.exterior.xy).T / float(scale_factor) for t in trees])
    min_x, min_y = xys.min(axis=0)
    max_x, max_y = xys.max(axis=0)
    side_length = max(max_x - min_x, max_y - min_y)
    return side_length**2 / n

print("Loading baseline scores...")

In [ ]:
# Calculate per-N scores
scores = {}
for n in range(1, 201):
    trees = load_trees_for_n(baseline_df, n)
    if len(trees) == n:
        scores[n] = calculate_score(trees, n)

print(f"Calculated scores for {len(scores)} N values")
print(f"Total score: {sum(scores.values()):.6f}")

# Find N values with highest scores (most room for improvement)
scores_sorted = sorted(scores.items(), key=lambda x: -x[1])
print("\nTop 20 N values by score (highest = most room for improvement):")
for n, score in scores_sorted[:20]:
    print(f"  N={n:3d}: {score:.6f}")

In [ ]:
# Theoretical minimum analysis
# The minimum score per N is approximately 0.355 (from eazy optimizer comments)
theoretical_min_per_n = 0.355

print("\n" + "=" * 60)
print("THEORETICAL ANALYSIS")
print("=" * 60)

theoretical_total = sum(theoretical_min_per_n for _ in range(1, 201))
print(f"Theoretical minimum (0.355 * 200): {theoretical_total:.2f}")
print(f"Current baseline total: {sum(scores.values()):.6f}")
print(f"Gap from theoretical: {sum(scores.values()) - theoretical_total:.2f}")
print()
print("Note: The theoretical minimum of 0.355 per N is an approximation.")
print("Actual minimum varies by N and may be lower for some N values.")

In [ ]:
# Analyze which N values are furthest from theoretical minimum
print("\n" + "=" * 60)
print("N VALUES FURTHEST FROM THEORETICAL MINIMUM")
print("=" * 60)

gaps = [(n, score - theoretical_min_per_n) for n, score in scores.items()]
gaps_sorted = sorted(gaps, key=lambda x: -x[1])

print("\nTop 20 N values with largest gap from theoretical minimum:")
for n, gap in gaps_sorted[:20]:
    print(f"  N={n:3d}: score={scores[n]:.6f}, gap={gap:.6f}")

print("\nBottom 20 N values (closest to theoretical minimum):")
for n, gap in gaps_sorted[-20:]:
    print(f"  N={n:3d}: score={scores[n]:.6f}, gap={gap:.6f}")

In [ ]:
# Summary and recommendations
print("\n" + "=" * 60)
print("SUMMARY AND RECOMMENDATIONS")
print("=" * 60)

print("""
1. VALIDATION PROBLEM:
   - 4 consecutive ensemble submissions failed Kaggle validation
   - Local overlap detection doesn't match Kaggle's
   - Need to use Kaggle's EXACT validation (scale_factor=1e18, intersects() and not touches())

2. LOCAL OPTIMUM PROBLEM:
   - Baseline is at a tight local optimum
   - bbox3, SA, fix_direction all found NO improvement
   - Need fundamentally different approach

3. GAP ANALYSIS:
   - Current: 70.676, Target: 68.887, Gap: 1.79 points (2.6%)
   - Small N values (1-20) have highest scores - most room for improvement
   - N=1 alone contributes 0.66 points (optimal is 0.35)

4. RECOMMENDED APPROACHES:
   a) Focus on small N values (1-30) where gap is largest
   b) Try eazy optimizer with multi-scale approach
   c) Implement fractional translation from jonathanchan kernel
   d) Use ultra-conservative validation before submission
""")

In [ ]:
# Check if we have eazy optimizer available
import os

eazy_paths = []
for root, dirs, files in os.walk('/home/nonroot/snapshots'):
    for f in files:
        if 'eazy' in f.lower():
            eazy_paths.append(os.path.join(root, f))

print("Eazy optimizer files found:")
for p in eazy_paths[:10]:
    print(f"  {p}")

if not eazy_paths:
    print("No eazy optimizer found - need to compile from source")